In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw09.ipynb")

<a id="top"></a>

# Homework 9:  PCA
## Due Date

This assignment is due **Thursday, November 17th at 11:59 PST**.

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** below.

**Collaborators**: *list collaborators here*

## Grading
Grading is broken down into autograded answers and free response. For autograded answers, the results of your code are compared to provided and/or hidden tests. For free response, readers will evaluate how well you answered the question and/or fulfilled the requirements of the question.

<!--
<details>
    <summary>[Click to Expand] <b>Scoring Breakdown</b></summary>-->
|Question|Points|
|---|---|
|Q1a | 1 |
|Q1b | 1 |
|Q1c | 1 |
|Q1d | 1 |
|Q1e | 2 |
|Q2a | 2 |
|Q2b | 2 |
|Q3a | 1 |
|Q3b | 2 |
|Q3c | 2 |
|Total | 15 |
</details>

In [ ]:
# Run this cell to set up your notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import sqlalchemy
from pathlib import Path

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
np.set_printoptions(threshold=5) # avoid printing out big matrices
%matplotlib inline

<br/><br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


# U.S. Presidential Elections By State

If you haven't worked with Principal Component Analysis before, we highly encourage you to take a look at the PCA lab first. PCA really shines on data where you have reason to believe that the data is relatively low in rank. 

We'll look at how states voted in presidential elections between 1972 and 2016. **Our ultimate goal is to show how 2D PCA scatterplots can allow us to identify clusters in a high dimensional dataset.** For this example, that means finding groups of states that vote similarly by plotting their 1st and 2nd principal components.

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 1

We explore a dataset on U.S. Presidential Elections by State since 1789, as taken from [Wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state).

In [ ]:
df = pd.read_csv("data/presidential_elections.csv")
df.head(5)

The data in this table is pretty messy (missing records, inconsistent field naming, etc.), so let's create a clean version.

Running the cell below will produce a clean table, which contains exactly 51 rows (corresponding to the 50 states plus Washington DC) and 13 columns (one for each of the election years from 1972 to 2020). The index of this dataframe is the state name.

In [ ]:
# just run this cell
df_clean = ( 
        df.iloc[:, -15:]    
        .drop(['Unnamed: 60'], axis = 1) 
        .rename(columns = {"2000 ‡": "2000", "2016 ‡": "2016", "State.1": "State"}) 
        .drop([51]) 
        .set_index("State")
)
df_clean

Side note: We produced the data cleaning function chain above by inspecting the CSV file. In your personal projects, you may be tempted to use Excel or Google Sheets (What You See Is What You Get, or WYSIWYG) to clean data. While sometimes more convenient, the downside of this is that you have no record of what you did—and if you have to redownload the data, you have to redo the manual data cleaning process.

<!-- BEGIN QUESTION -->

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 1a

What does each row in `df_clean` represent?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 1b

To perform PCA, we need to convert our data into numerical values. Create a `df_numerical` dataframe that replaces all of the "D" characters in `df_clean` with the number 0, and all of the "R" characters with the number 1. 

*Hint:* Use `df.replace` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)).

In [ ]:
df_numerical = ...

In [ ]:
grader.check("q1b")

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 1c

Now **standardize the data**: Center the data so that each column's mean is 0, and scale the data so that each column's variance is 1. Store your result in `df_standardized`.

In [ ]:
df_standardized = ...

In [ ]:
grader.check("q1c")

<br/>

We now have our data in a nice and tidy centered and scaled format. Phew! We are now ready to do PCA.

<br/>
<hr style="border: 1px solid #fdb515;" />

### Question 1d: SVD

In the following cell, compute the SVD of `df_standardized`:

$$\texttt{df}\_\texttt{standardized}=U\Sigma V^{T}$$


Store the $U$, $\Sigma$, and $V^T$ matrices in `u`, `s`, and `vt` respectively. This is one line of simple code (exactly like what we saw in lecture and what you did in lab) using the [`np.linalg.svd`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) function with the `full_matrices` argument set to `False`.

In [ ]:
...
u, s, vt

In [ ]:
grader.check("q1d")

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 1e: Get Principal Components

Using your results from the previous part, create a new `first_2_pcs` **dataframe** ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)) that contains exactly the first two columns of the principal components matrix. The first column should be labeled `pc1` and the second column should be labeled `pc2`. Store your result in `first_2_pcs`, and make sure to set the index to be the default numerical range index (i.e. 0, 1, 2, ...).

In [ ]:
first_2_pcs = ...
first_2_pcs.head()

In [ ]:
grader.check("q1e")

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 2

The cell below plots the 1st and 2nd principal components of our 50 states + Washington DC.

In [ ]:
# just run this cell
sns.scatterplot(data = first_2_pcs, x = "pc1", y = "pc2");

<!-- BEGIN QUESTION -->

Unfortunately, we have two problems:

1. There is a lot of overplotting, with only 28 distinct dots (out of 104 points). This means that at least some states voted exactly alike in these elections.
2. We don't know which state is which because the points are unlabeled.

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 2a: Jitter

Let's start by addressing problem 1. 

**In the cell below, create a new dataframe `first_2_pcs_jittered` with a small amount of random noise added to each principal component. In this same cell, create a scatterplot.**

To reduce overplotting, we **jitter** the first two principal components:
* Add a small amount of random, unbiased Gaussian noise to each value using `np.random.normal` ([documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) with mean 0 and standard deviation less than 1.
* Don't get caught up on the exact details of your noise generation; it's fine as long as your plot looks roughly the same as the original scatterplot, but without overplotting.
* The amount of noise you add *should not significantly affect* the appearance of the plot; it should simply serve to separate overlapping observations.

In [ ]:
# first, jitter the data
first_2_pcs_jittered = ...

# then, create a scatter plot
...

<!-- END QUESTION -->

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 2b

To address Problem 2, we can turn to Plotly. The below cell uses these Plotly guides ([example 1](https://plot.ly/python/text-and-annotations/), [example 2](https://plotly.com/python/hover-text-and-formatting/#disabling-or-customizing-hover-of-columns-in-plotly-express)) to create a scatter plot of the jittered .

In [ ]:
# just run this cell
import plotly.express as px

# get the state names from the standardized dataframe's index
first_2_pcs_jittered['state'] = df_standardized.index

# show state names on hover
fig = px.scatter(first_2_pcs_jittered, x="pc1", y="pc2",
                hover_data={"state": True}); 

fig.show(); 

<!-- BEGIN QUESTION -->

Analyze the above plot. In the below cell, address the following two points:
1. Give an example of a cluster of states that vote a similar way. Does the composition of this cluster surprise you? If you're not familiar with U.S. politics, it's fine to just say "No, I'm not surprised because I don't know anything about U.S. politics."
1. Include anything interesting that you observe. You will get credit for this as long as you write something reasonable that you can takeaway from the plot.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 3

We can also look at the contributions of each year's elections results on the values for our principal components.

Below, we define the `plot_pc` function that plots and labels the rows of $V^T$. We then call this function to plot the 1st row of $V^T$, i.e., the row of $V^T$ that corresponds to `pc1`.

**Note**: If you get an error when running this cell, make sure you are properly assigning the `vt` variable from Question 1.

In [ ]:
# just run this cell

def plot_pc(col_names, row_mat_vt, k):
    plt.bar(col_names, row_mat_vt[k, :], alpha=0.7)
    plt.xticks(col_names, rotation=90);
    
plt.figure(figsize=(12, 4)) # adjusts size of plot
plot_pc(list(df_standardized.columns), vt, 0);

<!-- BEGIN QUESTION -->


<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 3a

In the cell below, plot the the 2nd row of $V^T$, i.e., the row of $V^T$ that correpsonds to `pc2`.

In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->


<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 3b

Using the two above plots of the rows of $V^T$ as well as the original table, give a description of what it means for a point to be in the top-right quadrant of the 2-D scatter plot from Question 2.

In other words, what is generally true about a state with relatively large positive value for `pc1` (right side of 2-D scatter plot)? For a large positive value for `pc2` (top side of 2-D scatter plot)?

Notes:
* `pc2` is pretty hard to interpret, and the staff doesn't really have a consensus on what it means either. We'll be very nice when grading as long as your answer is reasonable - there is no correct answer necessarily. 
* Principal components beyond the first are often hard to interpret (but not always; see the lab).

_Type your answer here, replacing this text._

In [ ]:
# feel free to use this cell for scratch work. If you need more scratch space, add cells *below* this one.

# Make sure to put your actual answer in the cell above where it says "Type your answer here, replacing this text"

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br/>
<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 3c

To get a better sense of whether our 2D scatterplot captures the whole story, create a **scree plot** for this data. In other words, plot the fraction of the total variance (y-axis) captured by the ith principal component (x-axis).

*Hint:* Be sure to label your axes appropriately! You may find `plt.xticks()` ([documentation](https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.xticks.html)) helpful for formatting. Also check out the lab for more on scree plots.

In [ ]:
...

<!-- END QUESTION -->

<br/>

From your scree plot above, you should see that the first two principal components capture a large portion of the variance in our cleaned data. It is partially for this reason that the 2D scatter plot of principal components was easy to interpret.

## Congratulations!

Congrats! You are finished with this homework assignment.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)